# The BB84 QKD protocol


1. In the first step, Alice chooses two random bit strings, $k$ and $b$, that each consist of $n$ bits. Her bit string $k$ contains the actual bits she wants to encode (out of which the key will later be formed), while $b$ determines the bases in which she will encode her bits. For $b_i=0$ (i.e., if the $i^{th}$ bit is zero), she encodes the $i^{th}$ qubit in the standard $\{|0\rangle, |1\rangle \}$ basis, while for $b_i=1$, she encodes it in the $\{|+\rangle, |-\rangle \}$ basis, where $|+\rangle:=\frac{1}{\sqrt{2}}(|0\rangle +|1\rangle)$, $|-\rangle:=\frac{1}{\sqrt{2}}(|0\rangle -|1\rangle)$. This becomes more illustrative when representing each basis by two perpendicular arrows, where the two different bases are rotated by $45^\circ$. The encoding of each qubit $q_i$ would therefore look like the following:

<img align="center" width=300 src="./QKD-images/encoding_Alice.png">

2. After encoding her $n$ qubits, Alice sends these qubits to Bob. Bob also chooses a random bit string $\tilde{b}$ consisting of $n$ bits that determines in which bases he is going to perform measurements. He stores the outcomes of his measurements $\tilde{k_i}$ together with the corresponding basis bits $\tilde{b_i}$ in a table.

3. Next, Alice and Bob compare their basis bits $b_i$ and $\tilde{b}_i$. Whenever $b_i \neq \tilde{b}_i$, Bob measured in a different basis than Alice's qubit was encoded in, so he gets each outcome with probability $\frac{1}{2}$. Alice and Bob therefore discard all key bits corresponding to these basis bits. If $b_i = \tilde{b}_i$, however, they prepared and measured the qubit in the same basis, so (unless someone eavesdropped) Bob will get the key bit that Alice encoded, $\tilde{k}_i = k_i$. These outcomes then compose the key.

## An illustrated example
Suppose Alice's random bit strings are $k=`0111001`$ and $b=`1101000`$ and Bob's random bit string is $\tilde{b}=`1001101`$. Try to understand the other entries in the table below. Note that in the case where the basis bits are different, Bob has a 50% chance to get each outcome, so here one of them was chosen randomly.

<img src="./QKD-images/example_bb84.png" width=500 align="center">

The key produced in this example would be '0110'. To make sure that the key is secret and correct, Alice and Bob would "sacrifice" some of their key bits to check that no one eavesdropped. If someone had measured a qubit on the way, this could have changed the state of that qubit and with probability $\frac{1}{4}$, Bob's and Alice's key bits will be different. By checking $m$ bits, the probability to not notice an eavesdropper decreases as $\left(\frac{3}{4}\right)^m$. Thus, if they check enough bits and they are all the same, they can assume that no one eavesdropped and their key is secret. However, to keep things simple, we will not perfom these tests in this excercise. Instead, all bits of the key will be used.

### Message encrpytion
Once a secret key is distributed, Alice can encrypt her message by using the so-called one-time pad technique: she simply adds the key bits on top of her secret message bits that she wants to send. Using the example above, her key is $\text{key}=`0110`$. If her secret message bit string is $m=`1100`$, the encrypted message will be $c=m\oplus \text{key} = `1010`$. Bob can then decrypt the message by adding his key on that encrypted message, $m=c\oplus \text{key}$.

In [1]:
import random
from qiskit import *
from Alice import *
from Bob import *
from resources.teams import *
%matplotlib inline

ModuleNotFoundError: No module named 'qiskit'

## Step-1 & 2: Alice prepares the qubits and transmits. Bob measures the qubits.

In [66]:
from qiskit import QuantumCircuit
import random

random.seed(84) # DO NOT CHNAGE THIS SEED VALUE

alice_key = '11100100010001001001001000001111111110100100100100010010010001010100010100100100101011111110001010100010010001001010010010110010'

alice_bases = '11000110011000100001100101110000111010011001111111110100010111010100000100011001101010100001010010101011010001011001110011111111'

def alice_prepare_qubit(qubit_index):
    ## WRITE YOUR CODE HERE
    qc=QuantumCircuit(1,1)
    if alice_bases[qubit_index]==0:
        if alice_key==0:
            pass
        else:
            qc.x(0)
    else:
        
        if alice_bases[qubit_index]==0:
            qc.h(0)
        else:
            qc.x(0)
            qc.h(0)
                      
    return qc
           
    ## WRITE YOUR CODE HERE

In [74]:
def bob_measure_qubit(bob_bases, qubit_index, qubit_circuit):
    ## WRITE YOUR CODE HERE
    
    if bob_bases[qubit_index]==0: # Z basis
        qubit_circuit.measure(0,0)
    else: #X basis so change first and measure
        qubit_circuit.h(0)
        qubit_circuit.measure(0,0)
    
        
    ## WRITE YOUR CODE HERE

In [75]:
# Seeding a constant value to get uniform key accross all the teams
random.seed(84) # DO NOT CHNAGE THIS SEED IF YOU WANT TO COMPLETE THIS TASK XD

num_qubits = 130
bob_bases = str('{0:0100b}'.format(random.getrandbits(num_qubits)))

def bb84():
    print("Bob's bases:", bob_bases)
    
    all_qubit_circuits = []

    # now Alice sends her bits one by one
    for qubit_index in range(num_qubits-2):

        # Here Alice prepares the qubit
        ## WRITE YOUR CODE HERE
        this_qubit_circuit = alice_prepare_qubit(qubit_index)
        ## WRITE YOUR CODE HERE

        # Here Bob measures the Alice's qubit (Call the function for Bob to measure the qubits)
        ## WRITE YOUR CODE HERE
        bob_measure_qubit(bob_bases, qubit_index, this_qubit_circuit)
        ## WRITE YOUR CODE HERE

        # We collect all these qubits and put them in an array
        all_qubit_circuits.append(this_qubit_circuit)

    # Now execute all the circuits
    
    sim = Aer.get_backend('qasm_simulator')
    results = execute(all_qubit_circuits, backend=sim, shots=1).result()
 
    #And combine the results
    bits = ''
    for qubit_index in range(num_qubits-2):
        bits += [measurement for measurement in results.get_counts(qubit_index)][0]

    return bits
    
bits = bb84()
print("Bob's bits:", bits)

Bob's bases: 11000110011000000001100101110000111010011111111111110100000111010100100010011001001110100001010010111011010001011001111111011111
Bob's bits: 11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111


## Step-3: Create the shared key

The bases chosen by Alice are stored in the variable`alice_bases` in the file `Alice.py`. Use that information to now. compare the Alice's and Bob's bases to make the secret key!

In [35]:
print("Alice's bases:", alice_bases)
print("Bob's bases:", bob_bases)

def create_shared_key(alice_bases, bob_bases, bits):
    final_key = ''
    # Compare Alice and Bob's bases bits and generate the key by comparing if they're equal or not
    ## WRITE YOUR CODE HERE
    for q in range(len(bits)):
        if(alice_bases[q]==bob_bases[q]):
            final_key+=bits[q]
    ## WRITE YOUR CODE HERE
    return final_key
    
key = create_shared_key(alice_bases, bob_bases, bits)
print("Shared key:", key)

Alice's bases: 11000110011000100001100101110000111010011001111111110100010111010100000100011001101010100001010010101011010001011001110011111111
Bob's bases: 11000110011000000001100101110000111010011111111111110100000111010100100010011001001110100001010010111011010001011001111111011111
Shared key: 1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111


## Step-4: Transmit the message from Alice to Bob

In [89]:
plaintext_msg = '1101101001011010010010110011000101101101100110110110011010101000110110100101011001101011011001010100110101101011011'

def alice_send_message(message, key):
    # Generate ciphertext using the plaintext_msg and key and performing the EXOR operation
    ## WRITE YOUR CODE HERE
    ciphertext = ''
    for i in range(len(message)):
        ciphertext  = ciphertext+ str(int(message[i]) ^ int(key[i]))
    ## WRITE YOUR CODE HERE
    
    return ciphertext
    
sent_msg = alice_send_message(plaintext_msg, key)
print(sent_msg)

0010010110100101101101001100111010010010011001001001100101010111001001011010100110010100100110101011001010010100100


## Step-5: Bob receives and decodes Alice's message

In [90]:
def bob_decrypt_message(ciphertext, key):
    # get back plaintext using the sent_msg and key and performing the EXOR operation
    ## WRITE YOUR CODE HERE
    plaintext = ''
    for i in range(len(ciphertext)):
        plaintext = plaintext + str(int(ciphertext[i]) ^ int(key[i]))
    ## WRITE YOUR CODE HERE
    
    return plaintext

received_msg = bob_decrypt_message(sent_msg, key)
print(received_msg)

1101101001011010010010110011000101101101100110110110011010101000110110100101011001101011011001010100110101101011011


## Step-6: Additional Twist: Convert the received message from Binary encoded Morse code to ASCII text

In [94]:
# Use the below dictionary to decode the message that is encoded as morse code in binary.
MORSE_CODE_DICT = { 'a':'.-', 'b':'-...', 
                    'c':'-.-.', 'd':'-..', 'e':'.', 
                    'f':'..-.', 'g':'--.', 'h':'....', 
                    'i':'..', 'j':'.---', 'k':'-.-', 
                    'l':'.-..', 'm':'--', 'n':'-.', 
                    'o':'---', 'p':'.--.', 'q':'--.-', 
                    'r':'.-.', 's':'...', 't':'-', 
                    'u':'..-', 'v':'...-', 'w':'.--', 
                    'x':'-..-', 'y':'-.--', 'z':'--..', 
                    '1':'.----', '2':'..---', '3':'...--', 
                    '4':'....-', '5':'.....', '6':'-....', 
                    '7':'--...', '8':'---..', '9':'----.', 
                    '0':'-----', ', ':'--..--', '.':'.-.-.-', 
                    '?':'..--..', '/':'-..-.', '-':'-....-', 
                    '(':'-.--.', ')':'-.--.-', '!':'-.-.--'}

INV_MORSE = { v.replace('.', '10').replace('-', '110').strip('0'): k for k, v in MORSE_CODE_DICT.items()}
def convert_to_ascii_chars(received_msg):
    ## WRITE YOUR CODE HERE
    wordlist=received_msg.split('000')
    sentence=''
    for words in wordlist:
        word=''
        letters=words.split('00')
        for letter in letters:
            word=word+INV_MORSE[letter]
        sentence=sentence+" "+ word
    ascii_msg = sentence
    ## WRITE YOUR CODE HERE
    return ascii_msg

original_message = convert_to_ascii_chars(received_msg)
print('The message is:', original_message)

The message is:  great job guys!
